<a href="https://colab.research.google.com/github/abhishek-paghdal/Content_Generation/blob/main/BigScience_Bloom_560m_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#tf.config.set_visible_devices(tf.config.experimental.list_physical_devices('GPU')[0], 'GPU')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Major_Project/data.csv')

In [ ]:
df.head()

,_id,title,datePublished,description,article,keywords
0,{'$oid': '600837a2b5425d67582e0d8a'},EU lawmakers to push audio-visual sector on ge...,2020-11-30 05:46:35,European Union lawmakers are considering wheth...,European Union lawmakers are considering wheth...,"audio-visual, eu, geoblocking, Netflix, stream..."
1,{'$oid': '600837a3b5425d67582e0d8e'},Construction startup Scaled Robotics raises a ...,2020-02-03 07:13:18,"Industrial robots are expensive. But, then, so...","Industrial robots are expensive. But, then, so...","Battlefield, construction, Norwegian Construct..."
2,{'$oid': '600837a4b5425d67582e0d91'},Roblox buys digital avatar startup Loom.ai – T...,2020-12-14 11:10:15,Roblox announced today that it’s buying a digi...,Roblox announced today that it’s buying a digi...,"Avatar, Bitmoji, bitstrips, computing, films, ..."
3,{'$oid': '600837a4b5425d67582e0d92'},ClickUp hits $1 billion valuation in $100M Ser...,2020-12-15 16:30:14,Just six month after raising its first bit of ...,Just six month after raising its first bit of ...,"Android, ClickUp, Craft Ventures, Georgian Par..."
4,{'$oid': '600837a5b5425d67582e0d97'},"Spotify launches video podcasts worldwide, sta...",2020-07-21 07:13:39,Spotify today announced the global launch of v...,Spotify today announced the global launch of v...,"Spotify, streaming music, Video, YouTube"


In [ ]:
ndf.dropna(inplace=True)

In [ ]:
df1=df[:1000]

In [ ]:
df1.to_csv('data1.csv',index=False)

In [ ]:
data=pd.read_csv('data1.csv')

In [ ]:
data.shape

(1000, 6)

**Split the dataset into train and evaluate**

In [ ]:
from sklearn.model_selection import train_test_split

train_data, eval_data = train_test_split(data, test_size=0.2)

In [ ]:
train_data.shape

(800, 6)

In [ ]:
train_data.to_csv('train_data.csv')
eval_data.to_csv('eval_data.csv')


In [ ]:
import csv

# this is for the train data

# Open the CSV file and read the contents
with open('/content/train_data.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    data = [row['article'] for row in reader]

# Write the contents to a text file
with open('/content/train_dataset.txt', 'w', encoding='utf-8') as txtfile:
    txtfile.write('\n'.join(data))


In [ ]:
# import csv

# this is for the evaluate data

# Open the CSV file and read the contents
with open('/content/eval_data.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    data = [row['article'] for row in reader]

# Write the contents to a text file
with open('/content/eval_dataset.txt', 'w', encoding='utf-8') as txtfile:
    txtfile.write('\n'.join(data))


In [ ]:
#pip install transformers,accelarate

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bigscience/bloom-560m')


In [ ]:
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/train_dataset.txt',
    block_size=128 #The block_size parameter determines the maximum sequence length for each training example
)

In [ ]:
eval_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/eval_dataset.txt',
    block_size=128 #The block_size parameter determines the maximum sequence length for each training example
)

/usr/local/lib/python3.9/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

## Load the bigscience/bloom-560m model for fine-tuning:

In [ ]:
model = AutoModelForCausalLM.from_pretrained('bigscience/bloom-560m')

## Define training arguments

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',        # output directory
    num_train_epochs=3,            # total number of training epochs
    per_device_train_batch_size=4, # batch size per device during training
    save_steps=5000,              # number of training steps between checkpoints
    save_total_limit=2,           # limit the total amount of checkpoints to save
    prediction_loss_only=True,    # only compute loss for prediction (i.e., do not compute loss for input tokens)
    learning_rate=5e-5,
    warmup_steps=500,
    fp16=True,                     # use mixed precision training
    logging_dir='./logs',          # directory for storing logs
    logging_steps=1000,
    dataloader_num_workers=8,      # number of worker processes for data loading
)

In [ ]:
#create trainer object that starts finetuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1000,3.940000
2000,2.917500
3000,1.745100


TrainOutput(global_step=3534, training_loss=2.61682541679378, metrics={'train_runtime': 1576.8409, 'train_samples_per_second': 8.963, 'train_steps_per_second': 2.241, 'total_flos': 3281353958227968.0, 'train_loss': 2.61682541679378, 'epoch': 3.0})

## Generate text using the fine-tuned model:

In [ ]:
model.to('cuda')

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (

In [ ]:
prompt='Natural Language Processing'
generated_text = model.generate(
    input_ids = tokenizer.encode(prompt,return_tensors='pt').to('cuda'),
    max_length=100,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
)

# Generate text using the fine-tuned model

# generated_text = model.generate(
#     prompt = 'what is Artificial Intelligence',
#     input_ids = tokenizer.encode(prompt,return_tensors='pt').to('cuda'),
#     max_length=500,
#     do_sample=True,
#     top_k=50,
#     top_p=0.95,
#     temperature=1.0,
# )


In [ ]:
print(tokenizer.decode(generated_text[0], skip_special_tokens=False))

Natural Language Processing (LPNL) technology. The company is also using deep-reinforcement learning to enable the machine learning to detect things like faces and objects and automatically deliver relevant information to users based on activity. It also operates a marketplace that helps users discover and save products based on the best terms and conditions. The company is also using deep-reinforcement learning to enable the machine learning to detect things like faces and objects and automatically deliver relevant information to users based on activity. It also operates a


==========================================================================================================================================================================================================================================================================================================